For simplicity and education:
    functionality has been removed
        submission path
        versioning

In [87]:
import pandas as pd
import numpy as np

In [88]:
import os
dirname = os.getcwd()
print(dirname)

/home/jupyterlab/pipeline-202102


In [89]:
os.environ['MPLCONFIGDIR'] = dirname
import matplotlib

In [90]:
import xlsxwriter
import xlrd

In [91]:
from datetime import datetime
my_date = datetime.now()
print(my_date.isoformat())

2021-02-08T15:55:52.304064


In [92]:
print(my_date.strftime('%Y-%m-%dT%H:%M:%S.%f%z'))

2021-02-08T15:55:52.304064


In [93]:
runid = "R" + my_date.strftime('%Y%m%d%H%M%S%f%z')
runid

'R20210208155552304064'

In [94]:
filename = dirname + '/Data.xlsx'
filedf = pd.read_excel(filename,engine='openpyxl', sheet_name=None, header=None)

In [95]:
sheetnames = list(filedf.keys())

In [96]:
def validate_date(date_text):
    try:
        format_string = '%Y-%m-%d %H:%M:%S'
        datetime.strptime(date_text, format_string)
        return "OK"
    except ValueError:
        return None

def GetDataType(row_in):
    valueasstring = str(row_in['CellValue'])
    if validate_date(valueasstring) != None:
        return 'Date'
    elif valueasstring.isnumeric():
        return 'Number'
    else:
        return 'Text'

def add1_2row(row_in):
    return row_in['RowID'] + 1


def add1_2column(row_in):
    return row_in['ColumnID'] + 1

In [97]:
celldbstagedf = pd.DataFrame()
sheetdf = filedf['DataMap'].copy()
sheetdf["RowID"] = sheetdf.index
#sheetdf

In [98]:
for currentsheetname in sheetnames:

    sheetdf = filedf[currentsheetname].copy()
    sheetdf["RowID"] = sheetdf.index
    celldb = pd.melt(sheetdf, id_vars=['RowID'])
    celldb = celldb.rename(columns={"variable": "ColumnID", "value": "CellValue"})
    celldb["CellType"] = "Unset"
    celldb["FileName"] = runid
    celldb["SheetName"] = currentsheetname
    
    celldb["ColumnID"] = celldb["ColumnID"]

    try:
        celldb["CellType"] = celldb.apply(GetDataType, axis=1)
        celldb["RowID"] = celldb.apply(add1_2row, axis=1)
        celldb["ColumnID"] = celldb.apply(add1_2column, axis=1)

    except:

        celldb["CellType"] = 'unknown'
        celldb["RowID"] = 0
        celldb["ColumnID"] = 0

    celldbstagedf = celldbstagedf.append(celldb, ignore_index=True)

    celldb = pd.DataFrame()



In [99]:
fullcelldb = celldbstagedf.copy()

In [100]:
def GetTableFromCells(Dataset_in):
    Cells = pd.DataFrame()
    Cells = fullcelldb[(fullcelldb['SheetName'] == Dataset_in)].copy()
    Table = pd.DataFrame()
    Table = Cells.pivot_table(index = ['RowID'], values = 'CellValue',columns = 'ColumnID', aggfunc = 'max')
    Headings = Table.iloc[0]
    HeadingsList = list(Headings)
    HeadingsListTuple = enumerate(HeadingsList,start=1)
    HeadingsListDict = dict(HeadingsListTuple)
    Table = Table.rename(columns=HeadingsListDict)
    Table = Table.drop([1])
    Columns = Table.columns.tolist()
    return Table, Columns

In [101]:
MetaData_Table

ColumnID,Database,DataSetID,Schema,TableName
RowID,,,,
2,data,Ages,raw,Ages
3,data,Genders,raw,Genders
4,data,DoB,raw,DoB
5,data,MaritalStatus,raw,MaritalStatus
6,data,Department,raw,Department
7,data,Jobs,raw,Jobs
8,data,HomeLocation,raw,HomeLocation


In [102]:
Dataset_Table, Dataset_Columns = GetTableFromCells('DataSet')

In [103]:
Dataset_Table

ColumnID,DataSet,SheetName,ReferenceTextStart,ReferenceTextEnd,RowFromReferenceText,ColumnFromReferenceText,FieldPosition,FieldID,FieldDescription,TableOrientation
RowID,,,,,,,,,,
2,Ages,AgeGenderSheet,Age Table,Gender Table,2,1,1,Name,Name,Down
3,Ages,AgeGenderSheet,Age Table,Gender Table,2,2,2,Age,Age,Down
4,Genders,AgeGenderSheet,Gender Table,DOB Table,2,0,1,Name,Name,Down
5,Genders,AgeGenderSheet,Gender Table,DOB Table,2,1,2,Gender,Gender,Down
6,DoB,AgeGenderSheet,DOB Table,NaN,2,0,1,Name,Name,Down
7,DoB,AgeGenderSheet,DOB Table,NaN,2,1,2,DoB,D.O.B,Down
8,MaritalStatus,MaritalDepartmentSheet,Marital Status Table,NaN,2,0,1,Name,Name,NaN
9,MaritalStatus,MaritalDepartmentSheet,Marital Status Table,NaN,2,1,2,Status,Status,NaN
10,Department,MaritalDepartmentSheet,Department Table,NaN,2,0,1,Name,Name,NaN


In [104]:
ReferenceText_Table = Dataset_Table[['DataSet','SheetName','ReferenceTextStart']]
ReferenceText_Table = ReferenceText_Table.drop_duplicates()
ReferenceText_Table

ColumnID,DataSet,SheetName,ReferenceTextStart
RowID,,,
2,Ages,AgeGenderSheet,Age Table
4,Genders,AgeGenderSheet,Gender Table
6,DoB,AgeGenderSheet,DOB Table
8,MaritalStatus,MaritalDepartmentSheet,Marital Status Table
10,Department,MaritalDepartmentSheet,Department Table
12,Jobs,JobSheet,Job Description Table
15,HomeLocation,HomeSheet,Home Location Table


In [105]:
fullcelldb.head()

,RowID,ColumnID,CellValue,CellType,FileName,SheetName
0,1,1,NaN,Text,R20210208155552304064,AgeGenderSheet
1,2,1,NaN,Text,R20210208155552304064,AgeGenderSheet
2,3,1,NaN,Text,R20210208155552304064,AgeGenderSheet
3,4,1,NaN,Text,R20210208155552304064,AgeGenderSheet
4,5,1,NaN,Text,R20210208155552304064,AgeGenderSheet


In [106]:
## df1.merge(df2, left_on='lkey', right_on='rkey')
## new_df = pd.merge(A_df, B_df,  how='left', left_on=['A_c1','c2'], right_on = ['B_c1','c2'])

Anchor_Table = fullcelldb.merge(ReferenceText_Table,how='right', left_on=['SheetName','CellValue'], right_on=['SheetName','ReferenceTextStart'])

In [107]:
Anchor_Table = Anchor_Table.rename(columns={"RowID": "AnchorRowID", "ColumnID": "AnchorColumnID"})
Anchor_Table

,AnchorRowID,AnchorColumnID,CellValue,CellType,FileName,SheetName,DataSet,ReferenceTextStart
0,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table
1,14,4,Gender Table,Text,R20210208155552304064,AgeGenderSheet,Genders,Gender Table
2,23,1,DOB Table,Text,R20210208155552304064,AgeGenderSheet,DoB,DOB Table
3,3,1,Marital Status Table,Text,R20210208155552304064,MaritalDepartmentSheet,MaritalStatus,Marital Status Table
4,2,5,Department Table,Text,R20210208155552304064,MaritalDepartmentSheet,Department,Department Table
5,6,3,Job Description Table,Text,R20210208155552304064,JobSheet,Jobs,Job Description Table
6,8,1,Home Location Table,Text,R20210208155552304064,HomeSheet,HomeLocation,Home Location Table


In [108]:
Datamap_Table, Datamap_Columns = GetTableFromCells('DataMap')

In [109]:
Datamap_Table

ColumnID,SheetName,ReferenceText,RowFromReferenceText,ColumnFromReferenceText,TemplateText,DataGroup,DataCategory,DataScope,Column Header,DataSet
RowID,,,,,,,,,,
2,AgeGenderSheet,Age Table,2,1,Name,Ages,NaN,NaN,Name,Ages
3,AgeGenderSheet,Age Table,2,2,Age,Ages,NaN,NaN,Age,Ages
4,AgeGenderSheet,Gender Table,2,0,Name,Genders,NaN,NaN,Name,Genders
5,AgeGenderSheet,Gender Table,2,1,Gender,Genders,NaN,NaN,Gender,Genders
6,AgeGenderSheet,DOB Table,2,0,Name,DoB,NaN,NaN,Name,DoB
7,AgeGenderSheet,DOB Table,2,1,D.O.B,DoB,NaN,NaN,D.O.B,DoB
8,MaritalDepartmentSheet,Marital Status Table,2,0,Name,Marital Status,NaN,NaN,Name,MaritalStatus
9,MaritalDepartmentSheet,Marital Status Table,2,1,Status,Marital Status,NaN,NaN,Status,MaritalStatus
10,MaritalDepartmentSheet,Department Table,2,0,Name,Department,NaN,NaN,Name,Department


In [110]:
MetaData_Table, MetaData_Columns = GetTableFromCells('MetaData')

In [111]:
MetaData_Table

ColumnID,Database,DataSetID,Schema,TableName
RowID,,,,
2,data,Ages,raw,Ages
3,data,Genders,raw,Genders
4,data,DoB,raw,DoB
5,data,MaritalStatus,raw,MaritalStatus
6,data,Department,raw,Department
7,data,Jobs,raw,Jobs
8,data,HomeLocation,raw,HomeLocation


In [112]:
FullDataMap_Table = Anchor_Table.merge(
        Datamap_Table,how='right',
        left_on=['DataSet'],
        right_on=['DataSet'])
FullDataMap_Table

,AnchorRowID,AnchorColumnID,CellValue,CellType,FileName,SheetName_x,DataSet,ReferenceTextStart,SheetName_y,ReferenceText,RowFromReferenceText,ColumnFromReferenceText,TemplateText,DataGroup,DataCategory,DataScope,Column Header
0,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,2,1,Name,Ages,NaN,NaN,Name
1,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,2,2,Age,Ages,NaN,NaN,Age
2,14,4,Gender Table,Text,R20210208155552304064,AgeGenderSheet,Genders,Gender Table,AgeGenderSheet,Gender Table,2,0,Name,Genders,NaN,NaN,Name
3,14,4,Gender Table,Text,R20210208155552304064,AgeGenderSheet,Genders,Gender Table,AgeGenderSheet,Gender Table,2,1,Gender,Genders,NaN,NaN,Gender
4,23,1,DOB Table,Text,R20210208155552304064,AgeGenderSheet,DoB,DOB Table,AgeGenderSheet,DOB Table,2,0,Name,DoB,NaN,NaN,Name
5,23,1,DOB Table,Text,R20210208155552304064,AgeGenderSheet,DoB,DOB Table,AgeGenderSheet,DOB Table,2,1,D.O.B,DoB,NaN,NaN,D.O.B
6,3,1,Marital Status Table,Text,R20210208155552304064,MaritalDepartmentSheet,MaritalStatus,Marital Status Table,MaritalDepartmentSheet,Marital Status Table,2,0,Name,Marital Status,NaN,NaN,Name
7,3,1,Marital Status Table,Text,R20210208155552304064,MaritalDepartmentSheet,MaritalStatus,Marital Status Table,MaritalDepartmentSheet,Marital Status Table,2,1,Status,Marital Status,NaN,NaN,Status
8,2,5,Department Table,Text,R20210208155552304064,MaritalDepartmentSheet,Department,Department Table,MaritalDepartmentSheet,Department Table,2,0,Name,Department,NaN,NaN,Name
9,2,5,Department Table,Text,R20210208155552304064,MaritalDepartmentSheet,Department,Department Table,MaritalDepartmentSheet,Department Table,2,1,Department,Department,NaN,NaN,Department


In [114]:
FullDataPublish_Table = FullDataMap_Table.merge(
        fullcelldb,how='left',
        left_on=['SheetName_x'],
        right_on=['SheetName'])
FullDataPublish_Table

,AnchorRowID,AnchorColumnID,CellValue_x,CellType_x,FileName_x,SheetName_x,DataSet,ReferenceTextStart,SheetName_y,ReferenceText,...,DataGroup,DataCategory,DataScope,Column Header,RowID,ColumnID,CellValue_y,CellType_y,FileName_y,SheetName
0,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,...,Ages,NaN,NaN,Name,1,1,NaN,Text,R20210208155552304064,AgeGenderSheet
1,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,...,Ages,NaN,NaN,Name,2,1,NaN,Text,R20210208155552304064,AgeGenderSheet
2,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,...,Ages,NaN,NaN,Name,3,1,NaN,Text,R20210208155552304064,AgeGenderSheet
3,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,...,Ages,NaN,NaN,Name,4,1,NaN,Text,R20210208155552304064,AgeGenderSheet
4,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,...,Ages,NaN,NaN,Name,5,1,NaN,Text,R20210208155552304064,AgeGenderSheet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,8,1,Home Location Table,Text,R20210208155552304064,HomeSheet,HomeLocation,Home Location Table,HomeSheet,Home Location Table,...,Home Location,NaN,NaN,Location,8,2,NaN,Text,R20210208155552304064,HomeSheet
1172,8,1,Home Location Table,Text,R20210208155552304064,HomeSheet,HomeLocation,Home Location Table,HomeSheet,Home Location Table,...,Home Location,NaN,NaN,Location,9,2,Location,Text,R20210208155552304064,HomeSheet
1173,8,1,Home Location Table,Text,R20210208155552304064,HomeSheet,HomeLocation,Home Location Table,HomeSheet,Home Location Table,...,Home Location,NaN,NaN,Location,10,2,London,Text,R20210208155552304064,HomeSheet
1174,8,1,Home Location Table,Text,R20210208155552304064,HomeSheet,HomeLocation,Home Location Table,HomeSheet,Home Location Table,...,Home Location,NaN,NaN,Location,11,2,Liverpool,Text,R20210208155552304064,HomeSheet


In [123]:
def KeepColumn(ColumnID,ColumnFromReferenceText,AnchorColumnID):
    if ColumnID == ColumnFromReferenceText + AnchorColumnID:
        return 1
    return 0

def KeepRow(RowID,RowFromReferenceText,AnchorRowID):
    if RowID > RowFromReferenceText + AnchorRowID:
        return 1
    return 0
#dc.ColumnID = ds.ColumnFromReferenceText + @AnchorColumn

FullDataPublish_Table['KeepColumn'] = FullDataPublish_Table.apply(lambda x: KeepColumn(x['ColumnID'],x['ColumnFromReferenceText'],x['AnchorColumnID']), axis=1)
FullDataPublish_Table['KeepRow'] = FullDataPublish_Table.apply(lambda x: KeepRow(x['RowID'],x['RowFromReferenceText'],x['AnchorRowID']), axis=1)

##df.apply(lambda x: fxy(x['A'], x['B']), axis=1)


#dc.RowID > ds.RowFromReferenceText + @AnchorRow;
FullDataPublish_Table

,AnchorRowID,AnchorColumnID,CellValue_x,CellType_x,FileName_x,SheetName_x,DataSet,ReferenceTextStart,SheetName_y,ReferenceText,...,DataScope,Column Header,RowID,ColumnID,CellValue_y,CellType_y,FileName_y,SheetName,KeepColumn,KeepRow
0,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,...,NaN,Name,1,1,NaN,Text,R20210208155552304064,AgeGenderSheet,0,0
1,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,...,NaN,Name,2,1,NaN,Text,R20210208155552304064,AgeGenderSheet,0,0
2,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,...,NaN,Name,3,1,NaN,Text,R20210208155552304064,AgeGenderSheet,0,0
3,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,...,NaN,Name,4,1,NaN,Text,R20210208155552304064,AgeGenderSheet,0,0
4,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,...,NaN,Name,5,1,NaN,Text,R20210208155552304064,AgeGenderSheet,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,8,1,Home Location Table,Text,R20210208155552304064,HomeSheet,HomeLocation,Home Location Table,HomeSheet,Home Location Table,...,NaN,Location,8,2,NaN,Text,R20210208155552304064,HomeSheet,1,0
1172,8,1,Home Location Table,Text,R20210208155552304064,HomeSheet,HomeLocation,Home Location Table,HomeSheet,Home Location Table,...,NaN,Location,9,2,Location,Text,R20210208155552304064,HomeSheet,1,0
1173,8,1,Home Location Table,Text,R20210208155552304064,HomeSheet,HomeLocation,Home Location Table,HomeSheet,Home Location Table,...,NaN,Location,10,2,London,Text,R20210208155552304064,HomeSheet,1,0
1174,8,1,Home Location Table,Text,R20210208155552304064,HomeSheet,HomeLocation,Home Location Table,HomeSheet,Home Location Table,...,NaN,Location,11,2,Liverpool,Text,R20210208155552304064,HomeSheet,1,1


In [126]:
Data2BPublished_Cells = FullDataPublish_Table[(FullDataPublish_Table['KeepRow']==1) & (FullDataPublish_Table['KeepColumn']==1)]
Data2BPublished_Cells

,AnchorRowID,AnchorColumnID,CellValue_x,CellType_x,FileName_x,SheetName_x,DataSet,ReferenceTextStart,SheetName_y,ReferenceText,...,DataScope,Column Header,RowID,ColumnID,CellValue_y,CellType_y,FileName_y,SheetName,KeepColumn,KeepRow
61,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,...,NaN,Name,8,3,Ringo,Text,R20210208155552304064,AgeGenderSheet,1,1
62,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,...,NaN,Name,9,3,Brian,Text,R20210208155552304064,AgeGenderSheet,1,1
63,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,...,NaN,Name,10,3,NaN,Text,R20210208155552304064,AgeGenderSheet,1,1
64,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,...,NaN,Name,11,3,NaN,Text,R20210208155552304064,AgeGenderSheet,1,1
65,5,2,Age Table,Text,R20210208155552304064,AgeGenderSheet,Ages,Age Table,AgeGenderSheet,Age Table,...,NaN,Name,12,3,NaN,Text,R20210208155552304064,AgeGenderSheet,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,6,3,Job Description Table,Text,R20210208155552304064,JobSheet,Jobs,Job Description Table,JobSheet,Job Description Table,...,NaN,ID,10,5,2,Number,R20210208155552304064,JobSheet,1,1
1138,8,1,Home Location Table,Text,R20210208155552304064,HomeSheet,HomeLocation,Home Location Table,HomeSheet,Home Location Table,...,NaN,Employee,11,1,Ringo,Text,R20210208155552304064,HomeSheet,1,1
1139,8,1,Home Location Table,Text,R20210208155552304064,HomeSheet,HomeLocation,Home Location Table,HomeSheet,Home Location Table,...,NaN,Employee,12,1,Paul,Text,R20210208155552304064,HomeSheet,1,1
1174,8,1,Home Location Table,Text,R20210208155552304064,HomeSheet,HomeLocation,Home Location Table,HomeSheet,Home Location Table,...,NaN,Location,11,2,Liverpool,Text,R20210208155552304064,HomeSheet,1,1


In [ ]:
DROP TABLE IF EXISTS Working.PublishDataSet; 

select
--ds.[RowID]
ds.[ColumnFromReferenceText]
,ds.[DataSetID]
,ds.[FieldDescription]
,ds.[FieldID] as FieldName
,ds.[FieldPosition]
,ds.[ReferenceTextEnd]
,ds.[ReferenceTextStart]
,ds.[RowFromReferenceText]
,ds.[SheetName]
,ds.[TableOrientation]
--,dc.[SheetName]
,dc.[CellType]
,dc.[ColumnID]
,dc.[RowID]
,dc.[CellValue]
into Working.PublishDataSet from
working.DataSetTable ds, working.DataCellData dc
where
dc.SheetName = ds.SheetName and
dc.ColumnID = ds.ColumnFromReferenceText + @AnchorColumn and dc.RowID > ds.RowFromReferenceText + @AnchorRow;


--------------------- prepare for pivot -------------------------------------------------
DROP TABLE IF EXISTS Working.fieldnames; 

select FieldName into Working.fieldnames from Working.PublishDataSet;

set  @fieldnameslist = NULL

SELECT @fieldnameslist = ISNULL(@fieldnameslist + ',','') + QUOTENAME(FieldName)
FROM (SELECT DISTINCT FieldName FROM Working.fieldnames) AS FieldName;

DROP TABLE if exists Working.celldata; 
DROP TABLE if exists Working.pivoteddata;

--select 'DataSetDataSet';

--select * from Working.DataSetDataSet;


select * into Working.celldata from Working.PublishDataSet;

------- Do Pivot ----------

--Prepare the PIVOT query using the dynamic 
SET @DynamicPivotQuery = 
N'select * into Working.pivoteddata from (select RowID,FieldName,CellValue from Working.celldata) src
PIVOT(max(CellValue) for FieldName in ('+@fieldnameslist+')) piv;';
--Execute the Dynamic Pivot Query

--select @DynamicPivotQuery;

EXEC sp_executesql @DynamicPivotQuery;

---- Populate Pivoted Table

DROP TABLE IF EXISTS Working.PublishTable; 


In [ ]:
--------
set @createtable = NULL;

SELECT @createtable = ISNULL(@createtable + ',','') + CreateField
FROM (SELECT '['+ColumnName+']' + ' ' + ColumnType as CreateField FROM [Working].[DataMapTable]) AS CreateField;

SELECT @columnlist = ISNULL(@columnlist + ',','') + FieldName
FROM (SELECT '['+ColumnName+']' FieldName FROM [Working].[DataMapTable]) AS FieldName;

--SELECT ColumnName + ' ' + ColumnType as CreateField FROM [Working].[DataMapTable]

-- @TableSchema = [Schema]
-- @TableName = [TableName]

set @createtable = 'create table ' + @TableSchema +'.' + @TableName + '(' +@createtable + ')'

--select @createtable;

set @SQLCmnd = 'DROP TABLE IF EXISTS ' + @TableSchema +'.' + @TableName

EXEC sp_executesql @SQLCmnd;

EXEC sp_executesql @createtable;

set @SQLCmnd = 'insert into ' + @TableSchema +'.' + @TableName + ' select ' + @columnlist + ' from Working.pivoteddata;'

--select @SQLCmnd

EXEC sp_executesql @SQLCmnd;

DROP TABLE IF EXISTS Working.pivoteddata;

commit transaction;

--select * from Working.PublishTable;

GO